In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Button, Slider
import os
import time

def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

# Function to calculate the number of parent and daughter nuclei using the exact method
def exact_method(N0, lambd, t):
    N_parent = N0 * np.exp(-lambd * t)
    N_daughter = N0 * (1 - np.exp(-lambd * t))
    return N_parent, N_daughter

# Function to simulate the decay process using Monte Carlo method
def monte_carlo_simulation(N0, lambd, t_max, num_simulations=1000):
    # Precompute decay times for all nuclei
    decay_times = np.random.exponential(1/lambd, (num_simulations, N0))
    
    # Initialize arrays to store results
    time_points = np.linspace(0, t_max, 100)
    parent_counts = np.zeros(len(time_points))
    daughter_counts = np.zeros(len(time_points))
    std_parent = np.zeros(len(time_points))
    std_daughter = np.zeros(len(time_points))
    
    # Calculate counts at each time point
    for i, t in enumerate(time_points):
        decayed = np.sum(decay_times <= t, axis=1)
        parent_counts[i] = np.mean(N0 - decayed)
        daughter_counts[i] = np.mean(decayed)
        std_parent[i] = np.std(N0 - decayed)
        std_daughter[i] = np.std(decayed)
    
    return time_points, parent_counts, daughter_counts, std_parent, std_daughter, decay_times

def main():
    clear_screen()
    print("="*70)
    print("Interactive Nuclear Decay Simulation")
    print("Exact Method vs Monte Carlo Method with Animation")
    print("="*70)
    
    # Default parameters
    N0 = 100
    half_life = 5
    max_time = 30
    num_simulations = 500
    
    # Get user inputs
    try:
        N0 = int(input(f"Enter the initial number of parent nuclei (default {N0}): ") or N0)
        half_life = float(input(f"Enter the half-life in seconds (default {half_life}): ") or half_life)
        max_time = float(input(f"Enter the maximum time to simulate (default {max_time}): ") or max_time)
        num_simulations = int(input(f"Enter the number of Monte Carlo simulations (default {num_simulations}): ") or num_simulations)
        
        if N0 <= 0 or half_life <= 0 or max_time <= 0 or num_simulations <= 0:
            print("All values must be positive. Using defaults.")
            N0, half_life, max_time, num_simulations = 100, 5, 30, 500
    except ValueError:
        print("Invalid input. Using default parameters.")
        N0, half_life, max_time, num_simulations = 100, 5, 30, 500
    
    # Calculate decay constant
    lambd = np.log(2) / half_life
    
    # Create time array
    t = np.linspace(0, max_time, 100)
    
    # Calculate exact solution
    N_parent_exact, N_daughter_exact = exact_method(N0, lambd, t)
    
    # Run Monte Carlo simulation
    print("\nRunning Monte Carlo simulation...")
    mc_time, mc_parent, mc_daughter, std_parent, std_daughter, decay_times = monte_carlo_simulation(
        N0, lambd, max_time, num_simulations)
    
    # Set up the figure with subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12))
    plt.subplots_adjust(bottom=0.2, hspace=0.4)
    
    # Exact solution plot
    ax1.plot(t, N_parent_exact, 'b-', label='Parent Nuclei (Exact)', linewidth=2)
    ax1.plot(t, N_daughter_exact, 'r-', label='Daughter Nuclei (Exact)', linewidth=2)
    ax1.set_xlabel('Time (seconds)')
    ax1.set_ylabel('Number of Nuclei')
    ax1.set_title('Exact Solution')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, N0 * 1.1)
    
    # Monte Carlo results plot
    ax2.plot(mc_time, mc_parent, 'b-', label='Parent Nuclei (MC)', linewidth=2)
    ax2.plot(mc_time, mc_daughter, 'r-', label='Daughter Nuclei (MC)', linewidth=2)
    ax2.fill_between(mc_time, mc_parent - std_parent, mc_parent + std_parent, color='blue', alpha=0.2)
    ax2.fill_between(mc_time, mc_daughter - std_daughter, mc_daughter + std_daughter, color='red', alpha=0.2)
    ax2.set_xlabel('Time (seconds)')
    ax2.set_ylabel('Number of Nuclei')
    ax2.set_title('Monte Carlo Simulation with Standard Deviation')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, N0 * 1.1)
    
    # Animation plot
    ax3.set_xlim(-1.5, 1.5)
    ax3.set_ylim(-1.5, 1.5)
    ax3.set_title('Nuclear Decay Animation')
    ax3.axis('off')
    
    # Create positions for nuclei in a circle
    theta = np.linspace(0, 2*np.pi, N0)
    r = 1.2
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    # Create scatter plot for animation
    scatter = ax3.scatter(x, y, s=50, alpha=0.7, color='blue')
    
    # Add text boxes for parameters and counts
    param_text = ax3.text(-1.4, 1.3, '', fontsize=10, 
                         bbox=dict(facecolor='white', alpha=0.7))
    count_text = ax3.text(0.5, 1.3, '', fontsize=10,
                         bbox=dict(facecolor='white', alpha=0.7))
    time_text = ax3.text(0, -1.4, '', fontsize=12,
                        bbox=dict(facecolor='white', alpha=0.7))
    
    # Add time slider
    ax_slider = plt.axes([0.2, 0.05, 0.6, 0.03])
    time_slider = Slider(ax_slider, 'Time', 0, max_time, valinit=0)
    
    # Add buttons
    ax_play = plt.axes([0.2, 0.01, 0.1, 0.03])
    ax_pause = plt.axes([0.35, 0.01, 0.1, 0.03])
    ax_reset = plt.axes([0.5, 0.01, 0.1, 0.03])
    ax_step = plt.axes([0.65, 0.01, 0.1, 0.03])
    
    play_button = Button(ax_play, 'Play')
    pause_button = Button(ax_pause, 'Pause')
    reset_button = Button(ax_reset, 'Reset')
    step_button = Button(ax_step, 'Step')
    
    # Animation variables
    current_time = 0
    is_playing = False
    
    # Use first simulation for animation
    animation_decay_times = decay_times[0]
    
    # Update function for animation
    def update_animation():
        # Update exact solution plot
        for line in ax1.lines:
            if line.get_label().startswith('Current'):
                line.remove()
        
        ax1.axvline(x=current_time, color='g', linestyle='--', alpha=0.7, label='Current time')
        ax1.text(current_time + 0.1, N0 * 0.9, f't={current_time:.1f}s', 
                bbox=dict(facecolor='white', alpha=0.7))
        
        # Update Monte Carlo plot
        for line in ax2.lines:
            if line.get_label().startswith('Current'):
                line.remove()
        
        ax2.axvline(x=current_time, color='g', linestyle='--', alpha=0.7, label='Current time')
        
        # Update animation
        decayed_mask = (animation_decay_times <= current_time)
        parent_count = N0 - np.sum(decayed_mask)
        daughter_count = np.sum(decayed_mask)
        
        # Update colors
        colors = ['red' if decayed else 'blue' for decayed in decayed_mask]
        scatter.set_color(colors)
        
        # Update text
        param_text.set_text(f'N0 = {N0}\nHalf-life = {half_life}s\nλ = {lambd:.4f}s⁻¹')
        count_text.set_text(f'Parent: {parent_count}\nDaughter: {daughter_count}')
        time_text.set_text(f'Time: {current_time:.2f}s')
        
        # Redraw
        fig.canvas.draw_idle()
    
    # Animation function for FuncAnimation
    def animate(frame):
        nonlocal current_time
        if is_playing:
            current_time = min(current_time + 0.1, max_time)
            time_slider.set_val(current_time)
        update_animation()
        return scatter, param_text, count_text, time_text
    
    # Button callbacks
    def play(event):
        nonlocal is_playing
        is_playing = True
    
    def pause(event):
        nonlocal is_playing
        is_playing = False
    
    def reset(event):
        nonlocal current_time
        current_time = 0
        time_slider.set_val(0)
        update_animation()
    
    def step(event):
        nonlocal current_time
        current_time = min(current_time + max_time/20, max_time)
        time_slider.set_val(current_time)
        update_animation()
    
    # Slider callback
    def update_slider(val):
        nonlocal current_time
        current_time = val
        update_animation()
    
    # Connect callbacks
    play_button.on_clicked(play)
    pause_button.on_clicked(pause)
    reset_button.on_clicked(reset)
    step_button.on_clicked(step)
    time_slider.on_changed(update_slider)
    
    # Initial update
    update_animation()
    
    # Create animation
    ani = FuncAnimation(fig, animate, frames=int(max_time * 10), interval=100, blit=False, repeat=True)
    
    # Show the plot
    plt.show()
    
    # Display comparison table
    print("\n" + "="*60)
    print("COMPARISON TABLE")
    print("="*60)
    print(f"{'Time (s)':<10} {'Exact Parent':<15} {'Exact Daughter':<15} {'MC Parent':<15} {'MC Daughter':<15}")
    print("-" * 70)
    
    time_points = [0, half_life, max_time/2, max_time]
    for tp in time_points:
        exact_parent, exact_daughter = exact_method(N0, lambd, tp)
        # For MC, find the closest time point
        idx = np.abs(mc_time - tp).argmin()
        mc_parent_val = mc_parent[idx]
        mc_daughter_val = mc_daughter[idx]
        
        print(f"{tp:<10.1f} {exact_parent:<15.2f} {exact_daughter:<15.2f} "
              f"{mc_parent_val:<15.2f} {mc_daughter_val:<15.2f}")

if __name__ == "__main__":
    main()